### Goal of this notebook
Comparing Hudoc metadata with our annotations in terms of cited cases and articles.

In [62]:
# IMPORTANT NOTE: some cases in 'form-responses.csv' are annotated multiple times!

save = False

In [63]:
import re
from collections import Counter
import itertools
import numpy as np
import pandas as pd
import itertools
import time
import aiohttp
import asyncio

In [ ]:
df = pd.read_csv('form-responses.csv')
df = df[~df['ECLI number'].isna()]

# Get all unique ECLI numbers of our annotated cases
annotated_cases = pd.DataFrame(index=df['ECLI number'].unique())

# Assign Hudoc item id to cases based on ECLI
ecli_map = pd.read_pickle('echr-data/ecli_map.pickle')

def itemid(ecli):
    try:
        return ecli_map.loc[ecli]['itemid']
    except KeyError:
        return np.nan

annotated_cases['itemid'] = annotated_cases.index.map(itemid)

# See how many annotated cases have an Hudoc itemid
nan_cases = annotated_cases[annotated_cases['itemid'].isna()]

# Remove cases without Hudoc itemid
annotated_cases = annotated_cases[~annotated_cases['itemid'].isna()]

print(f"Annotated cases WITH Hudoc itemid: {len(annotated_cases)}")
print(f"Annotated cases WITHOUT Hudoc itemid: {len(nan_cases)}") # <-- e.g. (partly) inadmissible decisions

annotated_cases

In [106]:
# STATISTICS (approach):
#   1. Per case; get union of all cited articles, i.e. the union includes hudoc metadata and annotated articles.
#   2. See how complete the cited articles are (for both hudoc/annotations) by comparing them to the union

def statistics(hudoc_col, annotations_col, rm=None):

    stats = annotated_cases.copy(deep=True)

    for ecli, row in annotated_cases.iterrows():
        stats_hudoc = set(row[hudoc_col])
        stats_annotations = set(row[annotations_col])

        union = stats_hudoc.union(stats_annotations)

        if rm:
            union.remove(row[rm][0])

        int_hudoc = union.intersection(stats_hudoc)
        int_annot = union.intersection(stats_annotations)


        # % = % of all cited articles
        stats.at[ecli, '% hudoc'] = round(len(int_hudoc) / len(union) * 100)
        stats.at[ecli, '% annotations'] = round(len(int_annot) / len(union) * 100)

    stats['% hudoc'] = stats['% hudoc'].astype(int)
    stats['% annotations'] = stats['% annotations'].astype(int)

    print(f"Average hudoc: {round(stats['% hudoc'].mean())}%")
    print(f"Average annotations: {round(stats['% annotations'].mean())}%")

    return stats

In [ ]:
async def fetch_on_ecli(session, ecli: str, column: str):
    """ Fetch the corresponding application number(s) of an ECLI number"""
    ecli_encoded = ecli.replace(':', '%3A')
    url = f"https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((ecli%3A%22{ecli_encoded}%22))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,advopidentifier,advopstatus,ecli,appnoparts,sclappnos&sort=&start=0&length=20&rankingModelId=11111111-0000-0000-0000-000000000000"
    failed = False

    while True:
        async with session.get(url) as response:

            try:
                data = await response.json()
            except:
                print(f'{ecli} FAILED')
                failed = True
                time.sleep(1)
                continue

            if data['resultcount'] == 0:
                print(f'{ecli} NO RESULTS')
                return []

            if failed:
                print(f'{ecli} NOW SUCCEEDED')

            return [appno for appno in data['results'][0]['columns'][column].split(';')]

### Task: Compare articles

In [ ]:
# FYI: unique articles from all Hudoc cases
unique_articles = set()
ecli_map['article'].apply(lambda articles: [unique_articles.add(art) for art in articles if len(art)])
print(unique_articles)

In [66]:
# Populate our annotated cases with the cited articles from Hudoc metadata
annotated_cases['article_hudoc'] = annotated_cases.index.to_series().apply(lambda ecli: sorted(ecli_map.loc[ecli]['article']))

# Populate our annotated cases with the cited articles from our own annotations
annotated_article_references = pd.read_csv('ECLI_article_references.csv', index_col='ECLI number')

def articles_annotations(ecli):
    """ Get cited articles from our annotations """
    refs = annotated_article_references.loc[ecli]
    return sorted([col.lower() for col in annotated_article_references.columns if refs[col]])

annotated_cases['article_annotations'] = annotated_cases.index.to_series().apply(articles_annotations)

,itemid,article_hudoc,article_annotations,case_annotations,case_annotations_appno,appno,hudoc_annotations_appno
ECLI:CE:ECHR:2001:0118JUD002487694,001-59156,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[24876/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9..."
ECLI:CE:ECHR:2001:0118JUD002528994,001-59157,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 9063/80, 10843/84,...",[25289/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9..."
ECLI:CE:ECHR:2001:0118JUD002515494,001-59158,"[14, 36, 6, 8, p1-1, p1-2]","[14, 6, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[25154/94],"[19178/91, 20348/92, 10843/84, 7525/76, 9063/8..."
ECLI:CE:ECHR:2001:0118JUD002488294,001-59155,"[14, 36, 8]","[14, 8]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 10843/84, 16969/90...",[24882/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9..."
ECLI:CE:ECHR:1996:0916JUD002189393,001-58062,"[14, 18, 35, 46, 5, 6, 8, p1-1]","[14, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1961:0111DEC000078860, ECLI:CE:E...","[788/60, 5577/72, 5583/72, 5493/72, 5310/71, 7...",[21893/93],"[5493/72, 5310/71, 7654/76, 9697/82, 11889/85,..."
...,...,...,...,...,...,...,...
ECLI:CE:ECHR:2007:0215JUD004385498,001-79488,"[13, 14, 3, 8, p1-1]","[13, 14, 3, 8, p1-1]","[ECLI:CE:ECHR:1978:0118JUD000531071, ECLI:CE:E...","[5310/71, 9659/82, 9658/82, 13416/87, 21893/93...",[43854/98],[]
ECLI:CE:ECHR:1998:0424JUD002318494,001-58162,"[13, 14, 18, 2, 3, 41, 5, 6, 8, p1-1]","[13, 14, 2, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1989:0707JUD001403888, ECLI:CE:E...","[14038/88, 21893/93, 21987/93, 23878/94, 23879...","[23184/94, 23185/94]","[14038/88, 21893/93, 21987/93, 23878/94, 23186..."
ECLI:CE:ECHR:2012:0216JUD000908904,001-109119,"[6, 8, p1-1]","[13, 6, 8, p1-1]","[ECLI:CE:ECHR:1998:0224JUD002143993, ECLI:CE:E...","[21439/93, 48757/99, 75139/01, 8415/02, 19443/...",[9089/04],[]
ECLI:CE:ECHR:2016:1020JUD003400007,001-167557,"[13, 8]","[13, 8]","[ECLI:CE:ECHR:1988:0427JUD000965982, ECLI:CE:E...","[9659/82, 9658/82, 31365/96, 50963/99, 75139/0...",[34000/07],[]


In [79]:
print('>> Article Statistics')
statistics(hudoc_col='article_hudoc', annotations_col='article_annotations')

>> Article Statistics
Average hudoc: 95%
Average annotations: 78%


,itemid,article_hudoc,article_annotations,case_annotations,case_annotations_appno,appno,case_hudoc_appno,% hudoc,% annotations
ECLI:CE:ECHR:2001:0118JUD002487694,001-59156,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[24876/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9...",100,80
ECLI:CE:ECHR:2001:0118JUD002528994,001-59157,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 9063/80, 10843/84,...",[25289/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9...",100,80
ECLI:CE:ECHR:2001:0118JUD002515494,001-59158,"[14, 36, 6, 8, p1-1, p1-2]","[14, 6, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[25154/94],"[19178/91, 20348/92, 10843/84, 7525/76, 9063/8...",100,83
ECLI:CE:ECHR:2001:0118JUD002488294,001-59155,"[14, 36, 8]","[14, 8]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 10843/84, 16969/90...",[24882/94],"[20348/92, 10843/84, 7525/76, 9063/80, 16969/9...",100,67
ECLI:CE:ECHR:1996:0916JUD002189393,001-58062,"[14, 18, 35, 46, 5, 6, 8, p1-1]","[14, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1961:0111DEC000078860, ECLI:CE:E...","[788/60, 5577/72, 5583/72, 5493/72, 5310/71, 7...",[21893/93],"[5493/72, 5310/71, 7654/76, 9697/82, 11889/85,...",89,67
...,...,...,...,...,...,...,...,...,...
ECLI:CE:ECHR:2007:0215JUD004385498,001-79488,"[13, 14, 3, 8, p1-1]","[13, 14, 3, 8, p1-1]","[ECLI:CE:ECHR:1978:0118JUD000531071, ECLI:CE:E...","[5310/71, 9659/82, 9658/82, 13416/87, 21893/93...",[43854/98],[],100,100
ECLI:CE:ECHR:1998:0424JUD002318494,001-58162,"[13, 14, 18, 2, 3, 41, 5, 6, 8, p1-1]","[13, 14, 2, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1989:0707JUD001403888, ECLI:CE:E...","[14038/88, 21893/93, 21987/93, 23878/94, 23879...","[23184/94, 23185/94]","[14038/88, 21893/93, 21987/93, 23878/94, 23186...",100,80
ECLI:CE:ECHR:2012:0216JUD000908904,001-109119,"[6, 8, p1-1]","[13, 6, 8, p1-1]","[ECLI:CE:ECHR:1998:0224JUD002143993, ECLI:CE:E...","[21439/93, 48757/99, 75139/01, 8415/02, 19443/...",[9089/04],[],75,100
ECLI:CE:ECHR:2016:1020JUD003400007,001-167557,"[13, 8]","[13, 8]","[ECLI:CE:ECHR:1988:0427JUD000965982, ECLI:CE:E...","[9659/82, 9658/82, 31365/96, 50963/99, 75139/0...",[34000/07],[],100,100


### Task: Compare cases

NB: gebruik vanuit Hudoc de "sclappnos" kolom om geciteerde application numbers te vinden. Quote uit de manual:

Use the Strasbourg case-law field to search for judgments and decisions in which a particular Court or former Commission case has
been cited. The Court's reasoning is based on the reasoning given in earlier judgments and decisions.

In [37]:
# Populate our annotated cases with the cited cases from our own annotations
annotated_case_references = pd.read_csv('ECLI_case_references.csv', index_col='ECLI number')

def cases_annotations(ecli):
    """ Get cited articles from our annotations """
    refs = annotated_case_references.loc[ecli]
    return sorted([col for col in annotated_case_references.columns if refs[col]])

annotated_cases['case_annotations'] = annotated_cases.index.to_series().apply(cases_annotations)

,itemid,article_hudoc,article_annotations,case_annotations,case_annotations_appno,appno
ECLI:CE:ECHR:2001:0118JUD002487694,001-59156,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[24876/94]
ECLI:CE:ECHR:2001:0118JUD002528994,001-59157,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 9063/80, 10843/84,...",[25289/94]
ECLI:CE:ECHR:2001:0118JUD002515494,001-59158,"[14, 36, 6, 8, p1-1, p1-2]","[14, 6, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[25154/94]
ECLI:CE:ECHR:2001:0118JUD002488294,001-59155,"[14, 36, 8]","[14, 8]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 10843/84, 16969/90...",[24882/94]
ECLI:CE:ECHR:1996:0916JUD002189393,001-58062,"[14, 18, 35, 46, 5, 6, 8, p1-1]","[14, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1961:0111DEC000078860, ECLI:CE:E...","[788/60, 5577/72, 5583/72, 5493/72, 5310/71, 7...",[21893/93]
...,...,...,...,...,...,...
ECLI:CE:ECHR:2007:0215JUD004385498,001-79488,"[13, 14, 3, 8, p1-1]","[13, 14, 3, 8, p1-1]","[ECLI:CE:ECHR:1978:0118JUD000531071, ECLI:CE:E...","[5310/71, 9659/82, 9658/82, 13416/87, 21893/93...",[43854/98]
ECLI:CE:ECHR:1998:0424JUD002318494,001-58162,"[13, 14, 18, 2, 3, 41, 5, 6, 8, p1-1]","[13, 14, 2, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1989:0707JUD001403888, ECLI:CE:E...","[14038/88, 21893/93, 21987/93, 23878/94, 23879...","[23184/94, 23185/94]"
ECLI:CE:ECHR:2012:0216JUD000908904,001-109119,"[6, 8, p1-1]","[13, 6, 8, p1-1]","[ECLI:CE:ECHR:1998:0224JUD002143993, ECLI:CE:E...","[21439/93, 48757/99, 75139/01, 8415/02, 19443/...",[9089/04]
ECLI:CE:ECHR:2016:1020JUD003400007,001-167557,"[13, 8]","[13, 8]","[ECLI:CE:ECHR:1988:0427JUD000965982, ECLI:CE:E...","[9659/82, 9658/82, 31365/96, 50963/99, 75139/0...",[34000/07]


In [54]:
# For each annotated case, take the cited ECLI numbers and convert them to their corresponding application number(s).
# This is needed because we can then compare our annotation's application numbers with those of Hudoc.
case_appnumbers = []

async with aiohttp.ClientSession() as session:
    for _, row in annotated_cases.iterrows():

        cited_ecli_numbers = row['case_annotations']
        cited_appnumbers = await asyncio.gather(*[asyncio.ensure_future(fetch_on_ecli(session, nr, 'appno')) for nr in cited_ecli_numbers])
        case_appnumbers.append(list(itertools.chain(*cited_appnumbers)))

annotated_cases['case_annotations_appno'] = case_appnumbers

ECLI:CE:ECHR:1983:0224JUD000752576 FAILED
ECLI:CE:ECHR:1983:0224JUD000752576 NOW SUCCEEDED
ECLI:CE:ECHR:2009:1022JUD000357206 FAILED
ECLI:CE:ECHR:2009:1022JUD000357206 NOW SUCCEEDED
ECLI:CE:ECHR:1986:0507REP000965982 FAILED
ECLI:CE:ECHR:1986:0507REP000965982 NOW SUCCEEDED
ECLI:CE:ECHR:2000:1019JUD003110796 FAILED
ECLI:CE:ECHR:2000:1019JUD003110796 NOW SUCCEEDED
ECLI:CE:ECHR:2012:0424JUD005769310 FAILED
ECLI:CE:ECHR:2012:0424JUD005769310 NOW SUCCEEDED
ECLI:CE:ECHR:1997:0703JUD001361688 FAILED
ECLI:CE:ECHR:1997:0703JUD001361688 NOW SUCCEEDED
ECLI:CE:ECHR:2009:0115JUD002826106 FAILED
ECLI:CE:ECHR:2009:0115JUD002826106 NOW SUCCEEDED
ECLI:CE:ECHR:1998:0219JUD001496789 FAILED
ECLI:CE:ECHR:1998:0219JUD001496789 FAILED
ECLI:CE:ECHR:1998:0219JUD001496789 NOW SUCCEEDED
ECLI:CE:ECHR:1998:0424JUD00231849; NO RESULTS
ECLI:CE:ECHR:1997:0319JUD001835791 FAILED
ECLI:CE:ECHR:1997:0319JUD001835791 NOW SUCCEEDED
ECLI:CE:ECHR:2009:1022JUD000357206 FAILED
ECLI:CE:ECHR:2009:1022JUD000357206 NOW SUCCEEDED
EC

In [29]:
# Get appno for the index column.
# This is needed because we annotated based on ECLI number, but it should be based on application number.

async with aiohttp.ClientSession() as session:
    annotated_cases['appno'] = await asyncio.gather(*[asyncio.ensure_future(fetch_on_ecli(session, nr, 'appno')) for nr in list(annotated_cases.index)])

ECLI:CE:ECHR:2004:1026JUD003364696 FAILED
ECLI:CE:ECHR:2004:1118JUD005825500 FAILED
ECLI:CE:ECHR:2016:0913JUD007573713 FAILED
ECLI:CE:ECHR:2004:1026JUD003364696 NOW SUCCEEDED
ECLI:CE:ECHR:2004:1118JUD005825500 NOW SUCCEEDED
ECLI:CE:ECHR:2016:0913JUD007573713 NOW SUCCEEDED


In [85]:
# ONLY FOR THIS COMPARISON: remove rows with duplicate appplication numbers
annotated_cases = annotated_cases[annotated_cases['appno'].apply(len) == 1]

In [101]:
# For each annotated case, fetch the cited application numbers from Hudoc.
hudoc_appnumbers = []

async with aiohttp.ClientSession() as session:
    for ecli in list(annotated_cases.index):
        # hudoc_appnumbers.append(await fetch_on_ecli(session, ecli, 'sclappnos'))
        hudoc_appnumbers.append(await fetch_on_ecli(session, ecli, 'extractedappno'))

annotated_cases['case_hudoc_appno'] = hudoc_appnumbers

# TODO: remove own appno from cited appnos


In [107]:
print('>> Cited SCL Cases Statistics')
annotated_cases = statistics(hudoc_col='case_hudoc_appno', annotations_col='case_annotations_appno', rm='appno')
annotated_cases

>> Cited SCL Cases Statistics
Average hudoc: 68%
Average annotations: 88%


,itemid,article_hudoc,article_annotations,case_annotations,case_annotations_appno,appno,case_hudoc_appno,% hudoc,% annotations
ECLI:CE:ECHR:2001:0118JUD002487694,001-59156,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[24876/94],"[24876/94, 28/77, 57/78, 1/94, 18/94]",31,69
ECLI:CE:ECHR:2001:0118JUD002528994,001-59157,"[14, 36, 8, p1-1, p1-2]","[14, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 9063/80, 10843/84,...",[25289/94],"[25289/94, 28/77, 57/78, 1/94, 18/94]",29,71
ECLI:CE:ECHR:2001:0118JUD002515494,001-59158,"[14, 36, 6, 8, p1-1, p1-2]","[14, 6, 8, p1-1, p1-2]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 7525/76, 9063/80, 10843/84,...",[25154/94],"[25154/94, 28/77, 57/78, 1/94, 18/94]",24,76
ECLI:CE:ECHR:2001:0118JUD002488294,001-59155,"[14, 36, 8]","[14, 8]","[ECLI:CE:ECHR:1979:0613JUD000683374, ECLI:CE:E...","[6833/74, 7525/76, 9063/80, 10843/84, 16969/90...",[24882/94],"[24882/94, 28/77, 57/78, 1/94, 18/94]",29,71
ECLI:CE:ECHR:1996:0916JUD002189393,001-58062,"[14, 18, 35, 46, 5, 6, 8, p1-1]","[14, 3, 5, 6, 8, p1-1]","[ECLI:CE:ECHR:1961:0111DEC000078860, ECLI:CE:E...","[788/60, 5577/72, 5583/72, 5493/72, 5310/71, 7...",[21893/93],"[21893/93, 788/60, 5583/72, 10148/82, 1994/31]",29,86
...,...,...,...,...,...,...,...,...,...
ECLI:CE:ECHR:2007:0215JUD003785097,001-79486,"[13, 14, 18, 3, 8, p1-1]","[13, 14, 3, 6, 8, p1-1]","[ECLI:CE:ECHR:1978:0118JUD000531071, ECLI:CE:E...","[5310/71, 9659/82, 9658/82, 13416/87, 10588/83...",[37850/97],"[37850/97, 26973/95, 23423/94, 8811/02, 8813/0...",38,100
ECLI:CE:ECHR:2007:0215JUD004385498,001-79488,"[13, 14, 3, 8, p1-1]","[13, 14, 3, 8, p1-1]","[ECLI:CE:ECHR:1978:0118JUD000531071, ECLI:CE:E...","[5310/71, 9659/82, 9658/82, 13416/87, 21893/93...",[43854/98],"[43854/98, 26973/95, 23423/94, 8811/02, 8813/0...",35,100
ECLI:CE:ECHR:2012:0216JUD000908904,001-109119,"[6, 8, p1-1]","[13, 6, 8, p1-1]","[ECLI:CE:ECHR:1998:0224JUD002143993, ECLI:CE:E...","[21439/93, 48757/99, 75139/01, 8415/02, 19443/...",[9089/04],"[9089/04, 8415/02, 40450/04, 75139/01, 48757/9...",83,100
ECLI:CE:ECHR:2016:1020JUD003400007,001-167557,"[13, 8]","[13, 8]","[ECLI:CE:ECHR:1988:0427JUD000965982, ECLI:CE:E...","[9659/82, 9658/82, 31365/96, 50963/99, 75139/0...",[34000/07],"[34000/07, 18639/03, 31365/96, 50963/99, 21447...",86,93


In [109]:
for ecli, row in annotated_cases.sort_values(by='% hudoc').drop(columns=['itemid', 'article_hudoc', 'article_annotations', 'appno']).iterrows():
    print(ecli)
    print(f"HUDOC {row['% hudoc']}%  {sorted(row['case_hudoc_appno'])}")
    print(f"ANNOT {row['% annotations']}%  {sorted(row['case_annotations_appno'])}")
    print()

ECLI:CE:ECHR:1997:1128JUD002318694
HUDOC 0%  ['23186/94']
ANNOT 100%  ['21893/93', '21987/93', '23178/94', '5310/71']

ECLI:CE:ECHR:1999:0218JUD002951595
HUDOC 0%  ['29515/95']
ANNOT 100%  ['17371/90', '21890/93', '8695/79']

ECLI:CE:ECHR:2014:0612JUD001471704
HUDOC 6%  ['10200/04', '10373/05', '10911/05', '11706/09', '119/94', '128/08', '12959/05', '13221/08', '13378/05', '13466/03', '13488/07', '13751/02', '14163/04', '14717/04', '14902/04', '15189/10', '16021/02', '16091/90', '16472/04', '17056/06', '172/02', '17209/02', '1772/06', '18176/05', '18768/05', '19009/04', '19217/91', '19724/05', '19993/04', '20082/02', '21151/04', '2139/10', '21819/04', '2334/03', '237/97', '24098/94', '24790/04', '24876/94', '25701/94', '26736/04', '268/96', '27996/06', '28204/95', '28261/06', '28342/95', '28856/95', '29/98', '29515/95', '303/00', '31443/96', '32555/96', '33202/96', '34044/02', '34162/06', '34369/97', '34503/97', '34836/06', '35014/97', '35332/05', '3572/06', '36500/05', '36548/97', '37